In [1]:
import cv2
import numpy as np
net = cv2.dnn.readNet("C:\\Users\\TEJAS\\Downloads\\Hand-Gestures-Recognition-master\\Hand-Gestures-Recognition-master\\Yolo\\yolov3_custom_last.weights", "C:\\Users\\TEJAS\\Downloads\\Hand-Gestures-Recognition-master\\Hand-Gestures-Recognition-master\\Yolo\\yolov3_custom.cfg") 
classes = list()
f= open("C:\\Users\\TEJAS\\Downloads\\Hand-Gestures-Recognition-master\\Hand-Gestures-Recognition-master\\Yolo\\obj.names", "r") # read .names file
for line in f.readlines(): # loop each line in file
    classes.append(line.strip())  # split removes extra spaces and append the rest data into the list
layer_names = net.getLayerNames() # get layes names
layer=list() # init layes list
for i in net.getUnconnectedOutLayers():
    # we are getting the output layer to get detection of the object
    layer.append(layer_names[i - 1])
colors = np.random.uniform(0, 255, size=(len(classes), 3))
#provide video path in string or 0== internal webcam 2== external webcam
cap = cv2.VideoCapture(0) # read input from default. 0 is for webcam 1 is for video and 2 is for exteral webcam 
if (cap.isOpened()== False): 
    print("Error opening video stream or file")
while(cap.isOpened()):
    # get video Frame by Frame
    ret, frame = cap.read()
    if ret == True:
        img=frame#set img to frame.
        height, width,_ = img.shape #this will get image height and width 
        # create input blob to detect image
        #0.00392 is the scale factor
        #416,416 is the size we are passing to the algorithm
        # it basically prepares the input image to run through the deep neural network.
        #openCV works with BGR format  
        #so blob has 3 channels BGR 
        #416 * 416 image
        blobObject = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False) 
        net.setInput(blobObject) 
        outputs = net.forward(layer)#get next layer
        #this basically helps us to extract the bounding box 
        class_ids = list() #helps detect class label 
        confidences = list() #assurity level of what an object is lower the value means that the system couldn't detect it 0.5 is threshold value 
        boxes = list() #bounding box 
        #displaying information on the screen 
        for out in outputs:
            for detection in out: 
                scores = detection[5:]  #detection[5:] returns probability of each score 
                class_id = np.argmax(scores) #class id is associated with class list name which tells about the object
                confidence = scores[class_id] 
                if confidence > 0.5: #tresh >0.5 it goes from 0 to 1
                   #object detected
                   #get center 
                   #when object will be detected it will mark a O on it 
                    center_x = int(detection[0] * width) #returns center x 
                    center_y = int(detection[1] * height) #return center y
                    #get height and width 
                    w = int(detection[2] * width) #2 gives wudth
                    h = int(detection[3] * height) #3 returns height 
                    # get points for rectangle
                    x = int(center_x - w / 2) #top left x
                    y = int(center_y - h / 2) #top left y 
                    #combine makes top left object
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)
        dec = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)#number of dectected objects in frame
        for i in range(len(boxes)):
            if i in dec:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])#get label
                color = colors[i]#each classs has specific color defined above
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)#create rectangle
                cv2.putText(img, label, (10,50), cv2.FONT_ITALIC, 2, color, 3)
        cv2.imshow("HAND GESTURE", img)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()

In [2]:
# Imports
import numpy as np
import cv2
import math

# Open Camera
capture = cv2.VideoCapture(0)

while capture.isOpened():

    # Capture frames from the camera
    ret, frame = capture.read()

    # Get hand data from the rectangle sub window
    cv2.rectangle(frame, (100, 100), (300, 300), (0, 255, 0), 0)
    crop_image = frame[100:300, 100:300]

    # Apply Gaussian blur
    blur = cv2.GaussianBlur(crop_image, (3, 3), 0)

    # Change color-space from BGR -> HSV
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

    # Create a binary image with where white will be skin colors and rest is black
    mask2 = cv2.inRange(hsv, np.array([2, 0, 0]), np.array([20, 255, 255]))

    # Kernel for morphological transformation
    kernel = np.ones((5, 5))

    # Apply morphological transformations to filter out the background noise
    dilation = cv2.dilate(mask2, kernel, iterations=1)
    erosion = cv2.erode(dilation, kernel, iterations=1)

    # Apply Gaussian Blur and Threshold
    filtered = cv2.GaussianBlur(erosion, (3, 3), 0)
    ret, thresh = cv2.threshold(filtered, 127, 255, 0)

    # Show threshold image
    cv2.imshow("Thresholded", thresh)

    # Find contours
    image = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    try:
        # Find contour with maximum area
        contour = max(contours, key=lambda x: cv2.contourArea(x))

        # Create bounding rectangle around the contour
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(crop_image, (x, y), (x + w, y + h), (0, 0, 255), 0)

        # Find convex hull
        hull = cv2.convexHull(contour)

        # Draw contour
        drawing = np.zeros(crop_image.shape, np.uint8)
        cv2.drawContours(drawing, [contour], -1, (0, 255, 0), 0)
        cv2.drawContours(drawing, [hull], -1, (0, 0, 255), 0)

        # Find convexity defects
        hull = cv2.convexHull(contour, returnPoints=False)
        defects = cv2.convexityDefects(contour, hull)

        # Use cosine rule to find angle of the far point from the start and end point i.e. the convex points (the finger
        # tips) for all defects
        count_defects = 0

        for i in range(defects.shape[0]):
            s, e, f, d = defects[i, 0]
            start = tuple(contour[s][0])
            end = tuple(contour[e][0])
            far = tuple(contour[f][0])

            a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
            b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
            c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
            angle = (math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 180) / 3.14

            # if angle > 90 draw a circle at the far point
            if angle <= 90:
                count_defects += 1
                cv2.circle(crop_image, far, 1, [0, 0, 255], -1)

            cv2.line(crop_image, start, end, [0, 255, 0], 2)

        # Print number of fingers
        if count_defects == 0:
            cv2.putText(frame, "ONE", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255),2)
        elif count_defects == 1:
            cv2.putText(frame, "TWO", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        elif count_defects == 2:
            cv2.putText(frame, "THREE", (5, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        elif count_defects == 3:
            cv2.putText(frame, "FOUR", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        elif count_defects == 4:
            cv2.putText(frame, "FIVE", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        else:
            pass
    except:
        pass

    # Show required images
    cv2.imshow("Gesture", frame)
    all_image = np.hstack((drawing, crop_image))
    cv2.imshow('Contours', all_image)

    # Close the camera if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [4]:
!pip install opencv-python

  Using cached opencv_python-4.6.0.66-cp36-abi3-win_amd64.whl (35.6 MB)


In [2]:
import cv2
import numpy as np
net = cv2.dnn.readNet("D:\\downloads\\Hand-Gestures-Recognition\\Hand-Gestures-Recognition-master\\Yolo\\yolov3_custom_last.weights", "D:\\downloads\\Hand-Gestures-Recognition\\Hand-Gestures-Recognition-master\\Yolo\\yolov3_custom.cfg") 
#net = cv2.dnn.readNet("yolov3_custom_last.weights", "yolov3_custom.cfg")
classes = []

with open("D:\\downloads\\Hand-Gestures-Recognition\\Hand-Gestures-Recognition-master\\Yolo\\obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

colors = np.random.uniform(0, 255, size=(len(classes), 3))

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    
    height, width, channels = frame.shape
    
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    
    class_ids = []
    confidences = []
    boxes = []
    
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[class_ids[i]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    cv2.imshow("Hand Gesture Recognition", frame)
    
    if cv2.waitKey(1) == 27:
        break
        
cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:217: error: (-212:Parsing error) Failed to parse NetParameter file: D:\downloads\Hand-Gestures-Recognition\Hand-Gestures-Recognition-master\Yolo\yolov3_custom_last.weights in function 'cv::dnn::dnn4_v20221220::readNetFromDarknet'


In [1]:
import cv2
import numpy as np

# Load the pre-trained YOLO model
net = cv2.dnn.readNet("C:\\Users\\TEJAS\\Downloads\\Hand-Gestures-Recognition-master\\Hand-Gestures-Recognition-master\\Yolo\\yolov3_custom_last.weights", "C:\\Users\\TEJAS\\Downloads\\Hand-Gestures-Recognition-master\\Hand-Gestures-Recognition-master\\Yolo\\yolov3_custom.cfg") 

# Load the COCO class labels
classes = []
with open("C:\\Users\\TEJAS\\Downloads\\Hand-Gestures-Recognition-master\\Hand-Gestures-Recognition-master\\Yolo\\obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Get the names of the output layers
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Set the input image and scale it
img = cv2.imread("C:\\Users\\TEJAS\\Downloads\\demoimage.png")
height, width, channels = img.shape
scale = 0.00392
blob = cv2.dnn.blobFromImage(img, scale, (416, 416), (0, 0, 0), True, crop=False)

# Pass the input image through the network
net.setInput(blob)
outs = net.forward(output_layers)

# Extract information about detected objects
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = center_x - w / 2
            y = center_y - h / 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])

# Apply non-max suppression to remove overlapping boxes
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# Draw the final boxes on the input image
colors = np.random.uniform(0, 255, size=(len(classes), 3))
if len(indices) > 0:
    for i in indices.flatten():
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        confidence = str(round(confidences[i], 2))
        color = colors[class_ids[i]]
        cv2.rectangle(img, (int(x), int(y)), (int(x + w), int(y + h)), color, 2)
        cv2.putText(img, label + " " + confidence, (int(x), int(y) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Show the final output image
cv2.imshow("demoimage", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

IndexError: invalid index to scalar variable.